In [1]:
import import_hack
import steps
import data_store
from data_store.sqlite_table_datastore import SQLiteTableDataStore
from data_store.sqlite_table_one_to_many_datastore import SQLiteTableOneToManyDataStore
from data_store.file_system_directory_datastore import FileSystemDirectoryDataStore
from data_store.numpy_datastore import NumpyDataStore
from data_store.stream_ndarray_adapter_datastore import StreamNdarrayAdapterDataStore
from quantization.pq_quantizer import PQQuantizer, restore_from_clusters
from transformer import transformers as trs
from search.exhaustive_searcher import ExhaustiveSearcher
from search.inverted_multi_index_searcher import InvertedMultiIndexSearcher
from common.ds_utils import print_ds_items_info
from evaluation.retrieval_perfomance import PrecisionRecallAveragePrecisionEvaluator
from evaluation.ground_truth import BrodatzGroundTruth
import numpy as np
%load_ext autoreload
%autoreload 2

# Compute descriptors(siftsets)

In [2]:
img_dir_path = r'C:\data\images\brodatz\data.brodatz\size_213x213'
images_ds = FileSystemDirectoryDataStore(dir_path=img_dir_path)
print("images count in '{0}': ".format(img_dir_path), images_ds.get_count())
base_transformers=[trs.BytesToNdarray(), trs.NdarrayToOpencvMatrix()]

images count in 'C:\data\images\brodatz\data.brodatz\size_213x213':  999


In [3]:
histogram_transformers=base_transformers+[trs.OpencvMatrixToHistogram(True)]
histogram_ds= SQLiteTableDataStore("ds_data\imgid_histogram","imgid_histogram")
# steps.transform_step(images_ds, histogram_transformers,histogram_ds)
print_ds_items_info(histogram_ds)

count of items in ds:  999
shape of item[0]:  (256, 1)
shape of item[1]:  (256, 1)
shape of item[2]:  (256, 1)


In [4]:
lbphistogram_transformers=base_transformers+[trs.OpencvMatrixToLBPHistogram()]
lbphistogram_ds= SQLiteTableDataStore("ds_data\imgid_lbphistogram","imgid_lbphistogram")
# steps.transform_step(images_ds, lbphistogram_transformers, lbphistogram_ds)
print_ds_items_info(lbphistogram_ds)

count of items in ds:  999
shape of item[0]:  (512,)
shape of item[1]:  (512,)
shape of item[2]:  (512,)


# Exhaustive search by descriptors

In [12]:
n_nearest=25

In [117]:
# %%timeit -n3 -r3
histogram_neighbor_ids_ds=NumpyDataStore()
histogram_ndarray_ds = StreamNdarrayAdapterDataStore(histogram_ds, detect_final_shape_by_first_elem=True)
histogram_exs_searcher=ExhaustiveSearcher(histogram_ndarray_ds.get_items_sorted_by_ids(), histogram_ndarray_ds.get_ids_sorted())
steps.search_step(histogram_ds, histogram_exs_searcher, n_nearest , histogram_neighbor_ids_ds)

In [9]:
glcm_neighbor_ids_ds=NumpyDataStore()
glcm_ndarray_ds = StreamNdarrayAdapterDataStore(glcm_ds, detect_final_shape_by_first_elem=True)
glcm_exs_searcher=ExhaustiveSearcher(glcm_ndarray_ds.get_items_sorted_by_ids(), glcm_ndarray_ds.get_ids_sorted())
steps.search_step(glcm_ds, glcm_exs_searcher, n_nearest , glcm_neighbor_ids_ds)

In [118]:
# %%timeit -n3 -r3
lbphistogram_neighbor_ids_ds=NumpyDataStore()
lbphistogram_ndarray_ds = StreamNdarrayAdapterDataStore(lbphistogram_ds, detect_final_shape_by_first_elem=True)
lbphistogram_exs_searcher=ExhaustiveSearcher(lbphistogram_ndarray_ds.get_items_sorted_by_ids(), lbphistogram_ndarray_ds.get_ids_sorted())
steps.search_step(lbphistogram_ds,lbphistogram_exs_searcher, n_nearest , lbphistogram_neighbor_ids_ds)

In [11]:
print_ds_items_info(lbphistogram_neighbor_ids_ds, first_items_to_print=5, print_shape=False)

count of items in ds:  999
item[0]:  [  1   2   4   3   5   6   7   9 842 841 845 843 844   8 792 790 444 443
 442 846 567 369 785 962 780]
item[1]:  [  2   3   1   4   6   5   7 841 842 845   9 843 844 846 792 567 787   8
 961 560 444 830 790 559 839]
item[2]:  [  3   4   2   1   6   5   7 842   9 841 845 843 844 792   8 961 790 830
 567 258 261 846 962  84  90]
item[3]:  [  4   3   2   1   6   5   7   9 842 841 845   8 843 844 961 567 565  84
 566  90 792 443  88 564 507]
item[4]:  [  5   7   6   9   4   3   8   2   1 842 411 407 413 841 961  84 258 414
  56  88 369 408 410 507 360]


In [12]:
print_ds_items_info(histogram_neighbor_ids_ds, first_items_to_print=5, print_shape=False)

count of items in ds:  999
item[0]:  [  1   3   5   2   8   6   9   4   7 332 851 993 333 999 854 850 524 847
 996 848 853 992 852 855 331]
item[1]:  [  2   9   6   3   1   5   8   4   7 332 331  97 333 999  94 998 162 851
 993 524  19 995 854  26 839]
item[2]:  [  3   2   1   6   9   5   8   4   7 332 333 999 993 995 992 996  26 524
 851 331 998 854 847 850  19]
item[3]:  [  4   8   7   5   1   3   2   6   9 999 992 993 995 996 851 333 998 854
 847 848  26 524 850  25 852]
item[4]:  [  5   8   1   4   3   2   6   7   9 999 332 995 992 998 993 333 996 991
 994 851 524 162 854 847 848]


# Evaluate retrieval perfomance

In [13]:
perfomance_evaluator=PrecisionRecallAveragePrecisionEvaluator(BrodatzGroundTruth())

In [119]:
histogram_retrieval_perfomance_ds=NumpyDataStore()
steps.evaluation_step(histogram_neighbor_ids_ds, perfomance_evaluator, histogram_retrieval_perfomance_ds)

In [15]:
glcm_retrieval_perfomance_ds=NumpyDataStore()
steps.evaluation_step(glcm_neighbor_ids_ds, perfomance_evaluator, glcm_retrieval_perfomance_ds)

In [120]:
lbphistogram_retrieval_perfomance_ds=NumpyDataStore()
steps.evaluation_step(lbphistogram_neighbor_ids_ds, perfomance_evaluator, lbphistogram_retrieval_perfomance_ds)

In [17]:
print_ds_items_info(lbphistogram_retrieval_perfomance_ds, first_items_to_print=1, print_shape=False)
print_ds_items_info(histogram_retrieval_perfomance_ds, first_items_to_print=1, print_shape=False)

count of items in ds:  999
item[0]:  [[  1.           2.           3.           4.           5.           6.
    7.           8.           9.          10.          11.          12.
   13.          14.          15.          16.          17.          18.
   19.          20.          21.          22.          23.          24.
   25.        ]
 [  1.           1.           1.           1.           1.           1.
    1.           1.           0.88888889   0.8          0.72727273
    0.66666667   0.61538462   0.64285714   0.6          0.5625       0.52941176
    0.5          0.47368421   0.45         0.42857143   0.40909091
    0.39130435   0.375        0.36      ]
 [  0.1          0.2          0.3          0.4          0.5          0.6
    0.7          0.8          0.8          0.8          0.8          0.8
    0.8          0.9          0.9          0.9          0.9          0.9
    0.9          0.9          0.9          0.9          0.9          0.9
    0.9       ]
 [  0.1          0.2   

# Evaluate average retrieval perfomance

In [121]:
histogram_average_retrieval_perfomance_ds=NumpyDataStore()
steps.averaging_step(histogram_retrieval_perfomance_ds, histogram_average_retrieval_perfomance_ds)

In [19]:
glcm_average_retrieval_perfomance_ds=NumpyDataStore()
steps.averaging_step(glcm_retrieval_perfomance_ds,glcm_average_retrieval_perfomance_ds)

In [122]:
lbphistogram_average_retrieval_perfomance_ds=NumpyDataStore()
steps.averaging_step(lbphistogram_retrieval_perfomance_ds,lbphistogram_average_retrieval_perfomance_ds)

In [21]:
print_ds_items_info(histogram_average_retrieval_perfomance_ds, first_items_to_print=3, print_shape=False)

count of items in ds:  1
item[0]:  [[  1.           2.           3.           4.           5.           6.
    7.           8.           9.          10.          11.          12.
   13.          14.          15.          16.          17.          18.
   19.          20.          21.          22.          23.          24.
   25.        ]
 [  1.           0.97797798   0.95395395   0.92992993   0.90610611
    0.87771104   0.85113685   0.81969469   0.77922367   0.71741742
    0.66511967   0.62137137   0.58235158   0.54733305   0.51531532
    0.48736236   0.46311017   0.44088533   0.41999895   0.4007007
    0.38324038   0.36727637   0.35291814   0.33921421   0.32656657]
 [  0.1          0.1955956    0.28618619   0.37197197   0.45305305
    0.52662663   0.5957958    0.65575576   0.7013013    0.71741742
    0.73163163   0.74564565   0.75705706   0.76626627   0.77297297
    0.77977978   0.78728729   0.79359359   0.797998     0.8014014
    0.8048048    0.80800801   0.81171171   0.81411411   0.8

# Plotting

In [10]:
def plot_results(average_retrieval_perfomances_ds_arr, labels, i, ylabel, loc='upper right'):
    x_ds=StreamNdarrayAdapterDataStore(average_retrieval_perfomances_ds_arr[0], slice_get=(slice(None),0))
    results_ds_arr=[StreamNdarrayAdapterDataStore(arp_ds, slice_get=(slice(None),i)) for arp_ds in average_retrieval_perfomances_ds_arr]
    import matplotlib.pyplot as plt
    steps.plotting_step(x_ds, plt, labels,
                        results_ds_arr)
    plt.legend(loc=loc)
    plt.xlabel('n_nearest')
    plt.ylabel(ylabel)
    plt.yticks(np.linspace(0,1, 21))
    plt.xticks(x_ds.get_items_sorted_by_ids().ravel())
    plt.grid(True)
    plt.show()

In [11]:
labels=["histogram(bins:256)", "LBP histogram(r:1, points:8, uniform)",
        "histogram(inverted multi-index)", "LBP histogram(inverted multi-index)"]

In [100]:
plot_results([histogram_average_retrieval_perfomance_ds, glcm_average_retrieval_perfomance_ds,
              lbphistogram_average_retrieval_perfomance_ds], labels, 1,'precision', loc='lower left')

[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.]
[ 1.          0.97797798  0.95395395  0.92992993  0.90610611  0.87771104
  0.85113685  0.81969469  0.77922367  0.71741742  0.66511967  0.62137137
  0.58235158  0.54733305  0.51531532  0.48736236  0.46311017  0.44088533
  0.41999895  0.4007007   0.38324038  0.36727637  0.35291814  0.33921421
  0.32656657]
[ 1.          0.99449449  0.98531865  0.97272272  0.95955956  0.94577911
  0.92807093  0.90940941  0.88288288  0.80770771  0.74319774  0.68802135
  0.64179564  0.6006006   0.56516517  0.5325951   0.50391568  0.47747748
  0.45334808  0.43163163  0.41155441  0.39357539  0.3774209   0.36236236
  0.34846847]
[ 1.          0.93643644  0.88688689  0.8455956   0.80660661  0.77360694
  0.73902474  0.70720721  0.67534201  0.62612613  0.58312858  0.54562896
  0.51274351  0.48305448  0.45699032  0.43374625  0.41253018  0.39328217
  0.37532269  0.35955956  0.34491634  0.3

# Approximate search with inverted multi-index

### Quantize global descriptors

In [30]:
histogramclusters_ds=SQLiteTableDataStore('ds_data\imgid_histogramclusters256', 'clusterid_cluster')
quantizer = PQQuantizer(n_clusters=256, n_quantizers=2)
steps.quantize_step(histogram_ds, quantizer, histogramclusters_ds)
print_ds_items_info(histogramclusters_ds)

count of items in ds:  2
shape of item[0]:  (256, 128)
shape of item[1]:  (256, 128)


In [24]:
glcmclusters_ds=SQLiteTableDataStore('ds_data\imgid_glcmclusters', 'clusterid_cluster')
quantizer = PQQuantizer(n_clusters=256, n_quantizers=2)
# steps.quantize_step(glcm_ds, quantizer, glcmclusters_ds)
print_ds_items_info(glcmclusters_ds)

count of items in ds:  2
shape of item[0]:  (256, 32768)
shape of item[1]:  (256, 32768)


In [6]:
lbphistogramclusters_ds=SQLiteTableDataStore('ds_data\imgid_lbphistogramclusters50', 'clusterid_cluster')
quantizer = PQQuantizer(n_clusters=50, n_quantizers=2)
steps.quantize_step(lbphistogram_ds, quantizer, lbphistogramclusters_ds)
print_ds_items_info(lbphistogramclusters_ds)

count of items in ds:  2
shape of item[0]:  (50, 256)
shape of item[1]:  (50, 256)


### Build inverted multi-index

In [31]:
histogramclusters_ndarray_ds=StreamNdarrayAdapterDataStore(histogramclusters_ds, detect_final_shape_by_first_elem=True)
histgoramcluster_centers=histogramclusters_ndarray_ds.get_items_sorted_by_ids()
print("histgoramcluster_centers: ", histgoramcluster_centers.shape)
histogram_ndarray_ds = StreamNdarrayAdapterDataStore(histogram_ds, detect_final_shape_by_first_elem=True)
histogram_X = histogram_ndarray_ds.get_items_sorted_by_ids()
histogram_X_ids=histogram_ndarray_ds.get_ids_sorted()
print("X: ", histogram_X.shape)
print("X_ids: ", histogram_X_ids.shape)
histogram_imi_searcher=InvertedMultiIndexSearcher(histogram_X, histogram_X_ids, histgoramcluster_centers)

histgoramcluster_centers:  (2, 256, 128)
X:  (999, 256, 1)
X_ids:  (999,)


In [28]:
glcmclusters_ndarray_ds=StreamNdarrayAdapterDataStore(glcmclusters_ds, detect_final_shape_by_first_elem=True)
glcmcluster_centers=glcmclusters_ndarray_ds.get_items_sorted_by_ids()
glcm_ndarray_ds = StreamNdarrayAdapterDataStore(glcm_ds, detect_final_shape_by_first_elem=True)
glcm_X = glcm_ndarray_ds.get_items_sorted_by_ids()
glcm_X_ids=glcm_ndarray_ds.get_ids_sorted()
glcm_imi_searcher=InvertedMultiIndexSearcher(glcm_X, glcm_X_ids, glcmcluster_centers)

In [8]:
lbphistogramclusters_ndarray_ds=StreamNdarrayAdapterDataStore(lbphistogramclusters_ds, detect_final_shape_by_first_elem=True)
lbphistgoramcluster_centers=lbphistogramclusters_ndarray_ds.get_items_sorted_by_ids()
lbphistogram_ndarray_ds = StreamNdarrayAdapterDataStore(lbphistogram_ds, detect_final_shape_by_first_elem=True)
lbphistogram_X = lbphistogram_ndarray_ds.get_items_sorted_by_ids()
lbphistogram_X_ids=lbphistogram_ndarray_ds.get_ids_sorted()
lbphistogram_imi_searcher=InvertedMultiIndexSearcher(lbphistogram_X, lbphistogram_X_ids, lbphistgoramcluster_centers)

### Search approximate neighbors for all bovws(productsiftbincount_ds)

In [32]:
# %%timeit -n3 -r3
histogram_imi_neighborsids_ds=NumpyDataStore()
steps.search_step(histogram_ds, histogram_imi_searcher, n_nearest, histogram_imi_neighborsids_ds)

In [33]:
print_ds_items_info(histogram_imi_neighborsids_ds, print_shape=False, first_items_to_print=5)

count of items in ds:  999
item[0]:  [  3   1   8   5   9   2   6   4 332  98  97  95  94  92  91 331 998 997
 994 477 755 754 648 647 644]
item[1]:  [  9   2   6   3   1   8   5  98  97  95  94  92  91 332 477 331  99  96
  93 538 648 647 644 642 641]
item[2]:  [  3   1   9   2   6   8   5  98  97  95  94  92  91 332   4 477 331 998
 997 994 755 754 648 647 644]
item[3]:  [  4   7   8   5   3   1 998 997 994 995 991 332 755 754 999 992 261 648
 647 644 642 641 333 258 331]
item[4]:  [  8   5   3   1   4   9   2   6   7 332  98  97  95  94  92  91 998 997
 994 331 648 647 644 642 641]


In [21]:
glcm_approximateneighborsids_ds=NumpyDataStore()
steps.search_step(glcm_ds, glcm_imi_searcher, n_nearest, glcm_approximateneighborsids_ds)
print_ds_items_info(glcm_approximateneighborsids_ds, print_shape=False, first_items_to_print=5)

NameError: name 'glcm_ds' is not defined

In [154]:
# %%timeit -n3 -r3
lbphistogram_imi_neighborsids_ds=NumpyDataStore()
steps.search_step(lbphistogram_ds, lbphistogram_imi_searcher, n_nearest, lbphistogram_imi_neighborsids_ds)
# print_ds_items_info(lbphistogram_imi_neighborsids_ds, print_shape=False, first_items_to_print=5)

In [155]:
histogram_imi_retrieval_perfomance_ds=NumpyDataStore()
steps.evaluation_step(histogram_imi_neighborsids_ds, perfomance_evaluator, histogram_imi_retrieval_perfomance_ds)

In [156]:
lbphistogram_imi_retrieval_perfomance_ds=NumpyDataStore()
steps.evaluation_step(lbphistogram_imi_neighborsids_ds, perfomance_evaluator, lbphistogram_imi_retrieval_perfomance_ds)

In [157]:
histogram_imi_average_retrieval_perfomance_ds=NumpyDataStore()
steps.averaging_step(histogram_imi_retrieval_perfomance_ds, histogram_imi_average_retrieval_perfomance_ds)

In [158]:
lbphistogram_imi_average_retrieval_perfomance_ds=NumpyDataStore()
steps.averaging_step(lbphistogram_imi_retrieval_perfomance_ds, lbphistogram_imi_average_retrieval_perfomance_ds)

In [159]:
plot_results([histogram_average_retrieval_perfomance_ds, 
              lbphistogram_average_retrieval_perfomance_ds, histogram_imi_average_retrieval_perfomance_ds,
             lbphistogram_imi_average_retrieval_perfomance_ds], labels, 
             1, 'precision', loc='lower left')

[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.]
[ 1.          0.97797798  0.95395395  0.92992993  0.90610611  0.87771104
  0.85113685  0.81969469  0.77922367  0.71741742  0.66511967  0.62137137
  0.58235158  0.54733305  0.51531532  0.48736236  0.46311017  0.44088533
  0.41999895  0.4007007   0.38324038  0.36727637  0.35291814  0.33921421
  0.32656657]
[ 1.          0.93643644  0.88688689  0.8455956   0.80660661  0.77360694
  0.73902474  0.70720721  0.67534201  0.62612613  0.58312858  0.54562896
  0.51274351  0.48305448  0.45699032  0.43374625  0.41253018  0.39328217
  0.37532269  0.35955956  0.34491634  0.33092183  0.31844888  0.30697364
  0.29641642]
[ 0.97097097  0.94294294  0.91324658  0.88288288  0.85065065  0.81715048
  0.78149578  0.74537037  0.71249027  0.65515516  0.60578761  0.56456456
  0.52837453  0.49714     0.469336    0.44413163  0.42271683  0.40279168
  0.38480586  0.36806807  0.35244769  0.3

In [88]:
plot_results([histogram_average_retrieval_perfomance_ds, 
              lbphistogram_average_retrieval_perfomance_ds, histogram_imi_average_retrieval_perfomance_ds,
             lbphistogram_imi_average_retrieval_perfomance_ds], labels, 
             2, 
             'recall',
             loc='lower left')

[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.]
[ 0.1         0.1955956   0.28618619  0.37197197  0.45305305  0.52662663
  0.5957958   0.65575576  0.7013013   0.71741742  0.73163163  0.74564565
  0.75705706  0.76626627  0.77297297  0.77977978  0.78728729  0.79359359
  0.797998    0.8014014   0.8048048   0.80800801  0.81171171  0.81411411
  0.81641642]
[ 0.1         0.18728729  0.26606607  0.33823824  0.4033033   0.46416416
  0.51731732  0.56576577  0.60780781  0.62612613  0.64144144  0.65475475
  0.66656657  0.67627628  0.68548549  0.69399399  0.7013013   0.70790791
  0.71311311  0.71911912  0.72432432  0.72802803  0.73243243  0.73673674
  0.74104104]
[ 0.0970971   0.18858859  0.27397397  0.35315315  0.42532533  0.49029029
  0.54704705  0.5962963   0.64124124  0.65515516  0.66636637  0.67747748
  0.68688689  0.695996    0.704004    0.71061061  0.71861862  0.72502503
  0.73113113  0.73613614  0.74014014  0.7

In [104]:
plot_results([histogram_average_retrieval_perfomance_ds, 
              lbphistogram_average_retrieval_perfomance_ds, histogram_imi_average_retrieval_perfomance_ds,
             lbphistogram_imi_average_retrieval_perfomance_ds], labels,
             3, 'mean average precision',
             loc='lower left')

[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.]
[ 0.1         0.1955956   0.28575242  0.37031198  0.44955122  0.52087254
  0.58682421  0.64308046  0.68525597  0.6968876   0.70644261  0.71507625
  0.72165975  0.72646456  0.72972115  0.73303696  0.73662879  0.73943715
  0.74119681  0.7424931   0.74371337  0.74483267  0.74605564  0.74685226
  0.74755297]
[ 0.1         0.18728729  0.26459793  0.3344678   0.39624958  0.45293961
  0.50137375  0.54492981  0.580343    0.59308574  0.60261345  0.61007091
  0.61593832  0.62047857  0.62476286  0.6287043   0.63166019  0.63432953
  0.6363368   0.63834381  0.64005981  0.64122916  0.64261751  0.64388544
  0.64499455]
[ 0.0970971   0.18778779  0.27140474  0.34845679  0.41812646  0.47962129
  0.53281734  0.57781234  0.61768554  0.62692478  0.63366789  0.63974897
  0.64455377  0.64920128  0.65300508  0.65584542  0.65909573  0.66153706
  0.66387098  0.66573784  0.66715354  0.6